

# CoLA Library Exercise

This exercise is designed to help you get familiar with the CoLA linear algebra library. 

## Installation

Make sure you have a Python 3.10+ environment with either JAX or PyTorch installed. You can install CoLA using pip:



In [ ]:
%pip install git+https://github.com/wilson-labs/cola.git


Alternatively, you can open the documentation in Colab and start working from there: [Quick Start](https://colab.research.google.com/github/wilson-labs/cola/blob/master/docs/notebooks/colabs/Quick_Start.ipynb)

We strongly recommend that you read through the [documentation](https://cola.readthedocs.io/en/latest/index.html) to understand the library better.

## Basic Exercises
We'll start with some basic exercises to get you warmed-up for later. For each of the following you can use either JAX or PyTorch. We recommend that you try both and see if you spot any difference on the behavior.

1. Create a `LinearOperator` using the `ops.Diagonal` and `ops.Dense` classes. Perform basic operations like addition, subtraction, and multiplication on these operators. Also print the dense version of the `LinearOperator`. Verify that the computations are correct by using the dense API. 

2. Generate a random normal matrix $A$ of size $N=100$ and a random unitary vector $b$ and then solve the following linear system $Ax=b$ using `cola.inverse`. Compare the answers with `jnp.solve` or `torch.solve`. 

3. Get a random symmetric matrix $S$ of size $N=1,000$ by multiplying $S=A A^T$ where $A$ is a random normal matrix and a random unitary vector $b$ and solve $Sx=b$. Use `ops.PSD` to decorate $S$. Through `cola.inverse` find the solution. What algorithm is being use in this case? How can you modify some of the hyperparameters of these algorithm?

4. Find the eigenvalues and eigenvectors of the $T$ matrix constructed here: 
[Linear Operators: What and Why?](https://github.com/wilson-labs/cola/blob/main/docs/notebooks/LinOpIntro.ipynb). Compare the solutions obtained from decorating $T$ with `ops.Symmetric` and not decorating it. Are different algorithms being used? Is there a runtime benefit from dispatching a different algorithm?



## Large Scale Machine Learning with CoLA

Using JAX or PyTorch, pick any 3 out of the 5:

### 1. GP

GP Implement Gaussian Process (GP) inference with Radial Basis Function (RBF) kernel using `inverse()` from scratch on a dataset with at least 10k observations. You are not allowed to use GPyTorch. The formula for the GP posterior is:

$$f_* | X, y, X_* \sim \mathcal{N}(\mu_*, \Sigma_*)$$

where:

$$\mu_* = K(X_*, X)[K(X, X) + \sigma^2_n I]^{-1}y$$

$$\Sigma_* = K(X_*, X_*) - K(X_*, X)[K(X, X) + \sigma^2_n I]^{-1}K(X, X_*)$$

Here, $K$ is the RBF kernel, $X$ are the training inputs, $y$ are the training targets, $X_*$ are the test inputs, and $\sigma^2_n$ is the noise variance.



### 2. Hessian Spectrum
Compute the eigenspectrum of the Hessian of a pretrained neural network. You can download weights of image classifiers pretrained on CIFAR10. Use `cola.eig` or `cola.algorithms.lanczsos` and the spectral KDE smoothing method from [this paper](https://arxiv.org/pdf/1901.10159.pdf) to get a smoothed spectrum estimate.



### 3. Linear Regression
Implement linear regression with a heteroscedastic noise model where $\Phi$ is the design matrix, $\beta$ are the parameters and $\sigma_i$ is the measurement noise. The model is:

$$y = \Phi \beta + \epsilon, \quad \epsilon \sim \mathcal{N}(0, D)$$

where $D$ is a diagonal matrix with $\sigma_i^2$ on the diagonal. Add a Gaussian prior (regularization) if necessary.
    


Hint: $\hat{\beta}_{MLE} = (\Phi^T D^{-1} \Phi)^{-1} \Phi^T D^{-1} y$

### 4. Implement pagerank to find the most influential pages of Wikipedia.
 From the transition matrix on the [Linked- WikiText-2 dataset](https://rloganiv.github.io/linked-wikitext-2/#/), compute the largest eigenvector using `cola.eigmax`. From this eigenvector, rank the values to determine which web pages are most influential.

The PageRank algorithm computes the stationary distribution of a Markov chain. Given a transition matrix $P$, the PageRank vector $r$ is the eigenvector corresponding to the largest eigenvalue (which should be 1 for a stochastic matrix).

The transition matrix $P$ is defined as:

$$P = (1-\alpha)W + \alpha \mathbf{1}\mathbf{1}^T$$

where $W$ is the adjacency matrix normalized by the degree, $\alpha$ is the damping factor (usually set to 0.15), and $\mathbf{1}$ is a vector of ones.

The adjacency matrix $A_{ij}$ is 1 if there is a link from page $i$ to page $j$ (not the other way around). The degree-normalized adjacency matrix $W$ is obtained by dividing each row of $A$ by its sum.

The PageRank vector $r$ can be found by solving the eigenproblem:

$$P^T r = r$$

The entries of $r$ give the PageRank scores of the pages. The pages can then be ranked by these scores to find the most influential ones.


Below is some starter code to create an adjacency matrix. The pages are in the form of Wikipedia QIDs. After finding the most popular QIDs, if they are not in the `page_to_title dict`, you can look them up using the wikipedia API with the `get_titles_from_wikidata` function.

Suggestion: use the `cola.ops.Sparse` matrix for the adjacency matrix.

In [2]:
%pip --quiet install git+https://github.com/wilson-labs/cola.git
%pip install requests
%pip install io
%pip install zipfile

import json
import numpy as np
import requests
from collections import defaultdict
import requests, zipfile, io
r = requests.get("https://rloganiv.github.io/linked-wikitext-2/static/media/linked-wikitext-2.142e2e52.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

# Extract the JSONL file


# Initialize a dictionary to hold the adjacency list
adjacency_list = defaultdict(set)

# Initialize a dictionary to map page ids to indices
page_to_index = {}
index_to_page = {}
page_to_title = {}
next_index = 0


files = ['valid.jsonl', 'train.jsonl', 'test.jsonl']
for file in files:
  with z.open(file) as f:
      data = f.read().decode()
      for line in data.splitlines():
          data = json.loads(line)
          current_page_id = data['annotations'][0]['id']
          
          page_to_title[current_page_id] = data['title']
          # If the current page id is not in the dictionary, add it
          if current_page_id not in page_to_index:
              page_to_index[current_page_id] = next_index
              index_to_page[next_index] = current_page_id
              next_index += 1

          current_page_index = page_to_index[current_page_id]
          for annotation in data['annotations']:
              # If the annotation is a link to another page, add it to the adjacency list
              if annotation['source'] == 'WIKI' and annotation['id'] != current_page_id:
                  linked_page_id = annotation['id']

                  # If the linked page id is not in the dictionary, add it
                  if linked_page_id not in page_to_index:
                      page_to_index[linked_page_id] = next_index
                      index_to_page[next_index] = linked_page_id
                      next_index += 1

                  linked_page_index = page_to_index[linked_page_id]
                  adjacency_list[current_page_index].add(linked_page_index)

def get_titles_from_wikidata(qids):
    qids_string = '|'.join(qids)
    url = 'https://www.wikidata.org/w/api.php'
    params = {
        'action': 'wbgetentities',
        'ids': qids_string,
        'format': 'json',
        'props': 'labels',
        'languages': 'en'
    }
    response = requests.get(url, params=params)
    data = response.json()
    titles = {}
    for qid, entity in data['entities'].items():
        if 'en' in entity['labels']:
            titles[qid] = entity['labels']['en']['value']
    return titles

num_pages = len(page_to_index)
adjacency_matrix = np.zeros((num_pages, num_pages), dtype=int)
for page_index, linked_page_indices in adjacency_list.items():
    for linked_page_index in linked_page_indices:
        adjacency_matrix[page_index, linked_page_index] = 1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement io (from versions: none)
ERROR: No matching distribution found for io
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile
Note: you may need to restart the kernel to use updated packages.


In [4]:
print(adjacency_matrix.shape)

(45051, 45051)



### 5. Make a pull request to CoLA.
 e.g., improvement to the documentation, new commonly used linear operator (e.g., Fisher information matrix, banded matrix, FFT matrix), bug fix. If your code for one of the above exercises is particularly clean, consider adding markdown text explaining the steps and let's add it to the CoLA documentation under examples.
